In [1]:
! pip install pandas scikit-learn torch transformers tqdm nltk

# Install necessary packages
! pip install transformers evaluate scikit-learn seaborn imbalanced-learn torch

import numpy as np
import pandas as pd

from datasets import load_dataset
from datasets import ClassLabel, Value

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import AdamW

from transformers import BertTokenizer, BertModel
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import get_scheduler

import evaluate
import json
import pandas as pd
import seaborn as sns
from nltk import ngrams
from collections import Counter
import numpy as np

from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report

from tqdm.notebook import tqdm_notebook
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import torch
import json
import warnings
warnings.filterwarnings('ignore')

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


C:\Users\wrksy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
import pandas as pd
import json
import random
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, f1_score, classification_report
import copy
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, AutoModelForSequenceClassification, AdamW, get_scheduler
from tqdm.notebook import tqdm_notebook
import nltk

nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

# Define file paths for training and validation datasets
root_path = "C:/Users/wrksy/OneDrive/Desktop/Projects/7120CW2_code/data/"

# Function to preprocess data
def clean_data(data):
    cleaned_data = []
    for item in data:
        cleaned_item = {
            "uuid": item["uuid"],
            "postText": item["postText"],
            "targetTitle": item["targetTitle"],
            "targetParagraphs": item["targetParagraphs"],
            "spoiler": item["spoiler"] if "spoiler" in item else None
        }
        cleaned_data.append(cleaned_item)
    return cleaned_data

# Load and clean data
train_file_path = root_path + 'train.jsonl'
val_file_path = root_path + 'validation.jsonl'

with open(train_file_path,  encoding='utf-8') as train_file, open(val_file_path,  encoding='utf-8') as val_file:
    train_data = json.load(train_file)
    val_data = json.load(val_file)

clean_train_data = clean_data(train_data)
clean_val_data = clean_data(val_data)

# Flatten the dataset
def flatten_data(data, change_multi=False):
    flattened_data = []
    for item in data:
        uuid = item['uuid']
        post = item['postText'][0]
        target_title = item['targetTitle']
        target_paragraphs = item['targetParagraphs']
        spoilers = item['spoiler']
        
        for _idx, _target_paragraph in enumerate(target_paragraphs):
            spoiler_pos = None
            if spoilers and _target_paragraph in spoilers:
                spoiler_pos = _target_paragraph.index(_target_paragraph)
            _spoiler_type = None
            if change_multi and spoilers:
                spoiler_type = ["phrase", "passage", "multi"]
                _spoiler_texts = [spoiler["text"] for spoiler in spoilers]
                _max_length = max([len(_spoiler_text) for _spoiler_text in _spoiler_texts])
                _spoiler_type_idx = [len(_spoiler_text) for _spoiler_text in _spoiler_texts].index(_max_length)
                _spoiler_type = spoiler_type[_spoiler_type_idx]

            _data = {
                'uuid': uuid,
                'targetParagraphId': _idx,
                'postText': post,
                'targetTitle': target_title,
                'targetParagraph': _target_paragraph,
                'spoilerType': _spoiler_type if spoiler_pos else None,
                'spoilerPos': spoiler_pos if spoiler_pos else None
            }
            flattened_data.append(_data)

    return flattened_data

flattened_clean_train_data = flatten_data(clean_train_data, change_multi=True)
flattened_clean_val_data = flatten_data(clean_val_data, change_multi=True)

# Prepare final dataframe
def prepare_final_df(data):
    final_data_df = pd.DataFrame(data)
    final_data_df = final_data_df[~final_data_df['spoilerType'].isnull()].reset_index(drop=True)
    return final_data_df

train_data_df = prepare_final_df(flattened_clean_train_data)
val_data_df = prepare_final_df(flattened_clean_val_data)

print("Train dataset size:", len(train_data_df))
print("Validation dataset size:", len(val_data_df))

# Text augmentation
def synonym_replacement(words, n):
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word not in stop_words]))
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:
            break
    return new_words

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            synonym = l.name().replace("_", " ").replace("-", " ").lower()
            synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
            synonyms.add(synonym)
    if word in synonyms:
        synonyms.remove(word)
    return list(synonyms)

def augment_text(sentence, n_aug=1):
    words = sentence.split()
    augmented_sentences = []
    num_words = len(words)
    n_sr = max(1, int(0.1 * num_words))
    for _ in range(n_aug):
        a_words = synonym_replacement(words, n_sr)
        augmented_sentences.append(' '.join(a_words))
    return augmented_sentences

def augment_dataset(df, n_aug=1):
    augmented_data = []
    for index, row in df.iterrows():
        augmented_texts = augment_text(row['postText'], n_aug)
        for aug_text in augmented_texts:
            new_row = row.copy()
            new_row['postText'] = aug_text
            augmented_data.append(new_row)
    return pd.DataFrame(augmented_data)

# Data augmentation
augmented_train_df = augment_dataset(train_data_df, n_aug=1)

# Combine original and augmented data
train_data_df = pd.concat([train_data_df, augmented_train_df]).reset_index(drop=True)

# Encode labels
label_encoder = preprocessing.LabelEncoder()
train_data_df['spoilerType'] = label_encoder.fit_transform(train_data_df['spoilerType'])
val_data_df['spoilerType'] = label_encoder.transform(val_data_df['spoilerType'])

# BERT model for sequence classification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class SpoilerDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        post_text = str(self.data.postText[index])
        target_paragraph = str(self.data.targetParagraph[index])
        inputs = self.tokenizer(
            post_text,
            target_paragraph,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()
        label = torch.tensor(self.data.spoilerType[index], dtype=torch.long)
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label
        }

    def __len__(self):
        return self.len

train_dataset = SpoilerDataset(train_data_df, tokenizer, max_len=512)
val_dataset = SpoilerDataset(val_data_df, tokenizer, max_len=512)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8)

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_encoder.classes_))

# Use AdamW optimizer with weight decay
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=1e-2)

# Learning rate scheduler
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=len(train_dataloader) * 3
)

# Early stopping
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        elif val_loss >= self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# Training function
def train(model, train_dataloader, val_dataloader, optimizer, scheduler, num_epochs=3):
    early_stopping = EarlyStopping(patience=3, min_delta=0.01)
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = float('inf')
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        
        # Training phase
        model.train()
        train_loss = 0
        for batch in tqdm_notebook(train_dataloader):
            optimizer.zero_grad()
            outputs = model(input_ids=batch['input_ids'].to(device), attention_mask=batch['attention_mask'].to(device), labels=batch['labels'].to(device))
            loss = outputs.loss
            train_loss += loss.item()
            loss.backward()
            optimizer.step()
            scheduler.step()
        
        avg_train_loss = train_loss / len(train_dataloader)
        
        # Validation phase
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in tqdm_notebook(val_dataloader):
                outputs = model(input_ids=batch['input_ids'].to(device), attention_mask=batch['attention_mask'].to(device), labels=batch['labels'].to(device))
                loss = outputs.loss
                val_loss += loss.item()
        
        avg_val_loss = val_loss / len(val_dataloader)
        
        print(f'Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')
        
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
        
        early_stopping(avg_val_loss)
        if early_stopping.early_stop:
            print("Early stopping")
            break
    
    model.load_state_dict(best_model_wts)
    return model

# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Train the model
trained_model = train(model, train_dataloader, val_dataloader, optimizer, lr_scheduler, num_epochs=3)

# Evaluation function
def evaluate(model, val_dataloader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in tqdm_notebook(val_dataloader):
            outputs = model(input_ids=batch['input_ids'].to(device), attention_mask=batch['attention_mask'].to(device))
            logits = outputs.logits
            predictions.extend(torch.argmax(logits, dim=1).cpu().numpy())
            true_labels.extend(batch['labels'].cpu().numpy())
    return predictions, true_labels

# Evaluate the model
preds, true_labels = evaluate(trained_model, val_dataloader)

# Calculate metrics
accuracy = accuracy_score(true_labels, preds)
f1 = f1_score(true_labels, preds, average='weighted')
report = classification_report(true_labels, preds, target_names=label_encoder.classes_)

print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')
print(report)


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wrksy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\wrksy\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


JSONDecodeError: Extra data: line 2 column 1 (char 3995)